In [2]:
import pandas as pd

In [30]:
import dvc.api
import numpy as np

In [ ]:
# python main_inference_multimodal.py \
#         --model=LLM_Inference_Multimodal \
#         --model.llm_type="seqclf" \
#         --model.ckpt_path="models/product_title_multitask_multimodal/version_4/epoch=4-step=18868.ckpt" \
#         --model.config_path="models/product_title_multitask_multimodal/version_4/config.yaml" \
#         --model.task="seqclf_singlemodal_wishtitle2v121tax" \
#         --model.output_dir="models/product_title_multitask_multimodal/version_4" \
#         --model.write_interval="batch" \
#         --data=JSONListData \
#         --data.llm_type="seqclf" \
#         --data.label_map_file="datasets/taxonomy/wish_v1.2.1_newtax_allpaths.txt" \
#         --data.label_type="taxonomy" \
#         --data.data_source_yaml_path="datasets/multimodal_multitask/liang_product_offshorev2_multilabel_seqclf.yaml" \
#         --data.input_dict="{'template': 'Title: {title} ; Description: {description}', 'task_prefix': 'Classify product with description: '}" \
#         --data.output_dict="{'template': '{category_path}'}" \
#         --data.data_source_type="dvc" \
#         --data.model_name="t5-base" \
#         --data.batch_size=50 \
#         --data.max_length=50 \
#         --data.num_workers=0 \
#         --data.overwrite_cache=false \
#         --data.force_download_hfdata=false \
#         --trainer.logger=false \
#         --trainer.enable_checkpointing=false \
#         --trainer.accelerator gpu \
#         --trainer.strategy ddp

In [ ]:
# python main_inference_multimodal.py \
#         --model=LLM_Inference_Multimodal \
#         --model.llm_type="seqclf" \
#         --model.ckpt_path="models/product_title_multitask_multimodal/version_3/epoch=0-step=700.ckpt" \
#         --model.config_path="models/product_title_multitask_multimodal/version_3/config.yaml" \
#         --model.task="seqclf_singlemodal_wishtitle2v121tax" \
#         --model.output_dir="models/product_title_multitask_multimodal/version_3" \
#         --model.write_interval="batch" \
#         --data=JSONListData \
#         --data.llm_type="seqclf" \
#         --data.label_map_file="datasets/taxonomy/wish_v1.2.1_newtax_allpaths.txt" \
#         --data.label_type="taxonomy" \
#         --data.data_source_yaml_path="datasets/multimodal_multitask/wish_offshore_validated.yaml" \
#         --data.input_dict="{'template': '{title}', 'task_prefix': 'Classify product: '}" \
#         --data.output_dict="{'template': '{category}'}" \
#         --data.data_source_type="dvc" \
#         --data.model_name="t5-base" \
#         --data.batch_size=50 \
#         --data.max_length=50 \
#         --data.num_workers=0 \
#         --data.overwrite_cache=true \
#         --data.force_download_hfdata=true \
#         --trainer.logger=false \
#         --trainer.enable_checkpointing=false \
#         --trainer.accelerator gpu \
#         --trainer.strategy ddp

In [4]:
df_preds = pd.read_json('/workspaces/multitask-llm-rnd/modelling/models/product_title_multitask_multimodal/version_4/seqclf-epoch=4-step=18868--liang_product_offshorev2_multilabel_seqclf--test.json', 
                        lines=True)

In [5]:
df_preds.head(2)

,preds,batch_idx,dataloader_idx,batch_indices,logits
0,"[[shoes, 0.691628217697143], [shoes > women's ...",0,0,3,"[-8.90323257446289, -12.302337646484375, -13.2..."
1,"[[home & garden, 0.8693570494651791], [home & ...",0,0,10,"[-9.866235733032227, -12.931291580200195, -14...."


In [6]:
len(df_preds)

24810

In [7]:
# df_test = pd.read_json(dvc.api.get_url( 
#     'datasets/data/wish_products_internallabel/wish_products_offshore_labelled_validated_processed.json',
#     repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
# ), lines=True)

In [8]:
df_test = pd.read_json(dvc.api.get_url( 
    'datasets/data/liang_product/liang_product_classification_offshorev2only_test.json',
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
), lines=True)

In [9]:
df_test.head(2)

,title,description,category_path,category_id,pid,data_type
0,26 Style Unisex Men Women 2019 Ugly Christmas ...,"Suitable season: Spring,Autumn,winter Fabric n...",Women's Clothing > Sweaters > Sweaters,6037,5d8f06f627ffc134fe3d39da,test_v2_offshore
1,Gold Silver Western Texas Longhorn Buffalo Ste...,Condition:100% New Pattern Type:Animal Departm...,Apparel Accessories > Men's Ties & Handkerchiefs,71,58b458051f07f25129092ae9,test_v2_offshore


In [10]:
# df_test['category_path'] = df_test.category.apply(lambda x: ' > '.join(x))

In [11]:
len(df_test)

24810

In [12]:
df_preds = df_preds.sort_values('batch_indices').reset_index()

In [13]:
df_tax = pd.read_json(dvc.api.get_url(
    'datasets/data/taxonomy/wish_newtax.json',
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
), lines=True)

In [14]:
df_tax_leaf = df_tax[df_tax.is_leaf]

In [15]:
valid_leaf_paths = set(df_tax_leaf['category_path'].str.lower())

In [16]:
df_preds['preds'].apply(lambda x: x[0][0] in valid_leaf_paths).all()

False

In [17]:
def get_top_leaf_pred(x):
    for i in x:
        if i[0] in valid_leaf_paths:
            return i[0]
    return 'unknown'

In [31]:
logits = np.array(df_preds['logits'].tolist())

In [33]:
with open('../../datasets/taxonomy/wish_v1.2.1_newtax_allpaths.txt', 'r') as f:
    labs = f.read()

In [36]:
labs = [i.strip() for i in labs.split('\n') if len(i.strip()) > 0]

In [42]:
ind_labs = sorted([(ind, l) for ind, l in enumerate(labs) if l in valid_leaf_paths])

In [49]:
ind2lab = {ind: l for ind, l in ind_labs}

In [44]:
inds = np.array([i[0] for i in ind_labs])

In [51]:
df_preds['top_pred'] = [ind2lab[i] for i in inds[logits[:,inds].argmax(1)]]

In [18]:
# df_preds['top_pred'] = df_preds['preds'].apply(get_top_leaf_pred)

In [52]:
df_test.category_path.str.lower().apply(lambda x: x in valid_leaf_paths).all()

True

In [53]:
df_preds.top_pred.str.lower().apply(lambda x: x in valid_leaf_paths).all()

True

In [54]:
df_preds[~df_preds.top_pred.str.lower().apply(lambda x: x in valid_leaf_paths)].head(2)

,index,preds,batch_idx,dataloader_idx,batch_indices,logits,top_pred


In [55]:
from eval_from_file import perf_eval_util


In [56]:
df_test['category'] = df_test['category_path'].str.lower().apply(lambda x: x.split(' > '))
df_test['category_pred'] = df_preds['top_pred'].str.lower().apply(lambda x: x.split(' > '))

In [57]:
df_preds['top_pred']

0                    men's clothing > sweaters > pullovers
1         apparel accessories > men's ties & handkerchiefs
2        electronic components & supplies > passive com...
3                    shoes > women's shoes > women's flats
4        home & garden > festive & party supplies > eve...
                               ...                        
24805                     sports > fishing > fishing lines
24806    automobiles & motorcycles > motorcycle accesso...
24807           women's clothing > tops > blouses & shirts
24808             watches > men's watches > quartz watches
24809    jewelry & accessories > wedding & engagement j...
Name: top_pred, Length: 24810, dtype: object

In [58]:
df_test

,title,description,category_path,category_id,pid,data_type,category,category_pred
0,26 Style Unisex Men Women 2019 Ugly Christmas ...,"Suitable season: Spring,Autumn,winter Fabric n...",Women's Clothing > Sweaters > Sweaters,6037,5d8f06f627ffc134fe3d39da,test_v2_offshore,"[women's clothing, sweaters, sweaters]","[men's clothing, sweaters, pullovers]"
1,Gold Silver Western Texas Longhorn Buffalo Ste...,Condition:100% New Pattern Type:Animal Departm...,Apparel Accessories > Men's Ties & Handkerchiefs,71,58b458051f07f25129092ae9,test_v2_offshore,"[apparel accessories, men's ties & handkerchiefs]","[apparel accessories, men's ties & handkerchiefs]"
2,New Quality 1PC 12000uF 63V Amplifier/Audio/Po...,Features: 100% Brand new and high quality. ...,Electronic Components & Supplies > Passive Com...,2122,5d11be0228f36e1d1d3d65be,test_v2_offshore,"[electronic components & supplies, passive com...","[electronic components & supplies, passive com..."
3,2019 New Women Shoes Round Toe Flat Bottom Mot...,Welcome to our store! :) Women Shoes Size: U...,Shoes > Women's Shoes > Women's Pumps,4784,5d87170bd54a8c3c7401b476,test_v2_offshore,"[shoes, women's shoes, women's pumps]","[shoes, women's shoes, women's flats]"
4,Whiskey Bottle Beer Mug Super Shape Mylar Foil...,You will receive 2 pcs balloons.The pakage inc...,Home & Garden > Festive & Party Supplies > Eve...,2502,62230f6c1aec6fad16133bb2,test_v2_offshore,"[home & garden, festive & party supplies, even...","[home & garden, festive & party supplies, even..."
...,...,...,...,...,...,...,...,...
24805,FISHLUND Braided Fishing Line 100M 300M 500M F...,FISHLUND Fishing Line offers 15 variety of co...,Sports > Fishing > Fishing Lines,5033,62383e08b88420e2246f9ab7,test_v2_offshore,"[sports, fishing, fishing lines]","[sports, fishing, fishing lines]"
24806,"7/8"" 22mm Short 3D CNC Motorcycle Adjustable B...",Features: Upgrade the look with top quality le...,Automobiles & Motorcycles > Motorcycle Accesso...,729,612c8f50e8413e3f7969716d,test_v2_offshore,"[automobiles & motorcycles, motorcycle accesso...","[automobiles & motorcycles, motorcycle accesso..."
24807,ZANZEA Cotton Blouse Oversized Women Solid Col...,Package included:1 Blouse Material:100% Cotto...,Women's Clothing > Tops > Blouses & Shirts,6049,620ca17455f3999d7556d235,test_v2_offshore,"[women's clothing, tops, blouses & shirts]","[women's clothing, tops, blouses & shirts]"
24808,Men's Automatic Quartz Watch Personality Watch...,Movement type: automatic quartz Gender: male T...,Watches > Men's Watches > Quartz Watches,5975,61600766a058c58cd1d5128b,test_v2_offshore,"[watches, men's watches, quartz watches]","[watches, men's watches, quartz watches]"


In [60]:
perf_eval_util(df_test, 1, 'category_pred')

,id,precision,recall,f1-score,support,level
0,apparel accessories,0.887019,0.923655,0.904966,799.00000,1
1,automobiles & motorcycles,0.937403,0.950682,0.943996,1906.00000,1
2,beauty & health,0.929515,0.948315,0.938821,1335.00000,1
3,cellphones & telecommunications,0.939110,0.959330,0.949112,836.00000,1
4,computer & office,0.898917,0.895683,0.897297,278.00000,1
5,consumer electronics,0.867384,0.827350,0.846894,585.00000,1
6,education & office supplies,0.867550,0.715847,0.784431,366.00000,1
7,electronic components & supplies,0.714286,0.714286,0.714286,28.00000,1
8,entertainment,0.960265,0.868263,0.911950,167.00000,1
9,food,0.750000,0.705882,0.727273,17.00000,1
